<a href="https://colab.research.google.com/github/BRIJNANDA1979/BIGEARTHNET/blob/main/BigEarthNet_CNN_for_uniquely_labelled_46_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BigEarthNet unique labelled data with 46 classes uploaded on google drive.
Step  1: to create geotiff image using 12 bands of Sentinel-2 imagery **bold text**

In [15]:
import os
import pandas as pd

In [16]:
src_path = '/content/drive/MyDrive/BigEarthNet unique labelled'

In [11]:
!pip install rasterio

In [18]:
!pip install glob2

In [27]:
import rasterio
from rasterio import merge
import glob2
def merge_bands(path,sub):
    search = '*_B*.tif'   # 12 Band files end with B01,B02....
    files_batch = []
    files_batch = glob2.glob(os.path.join(path,search)) #search all tif band files 
    files_batch =sorted(files_batch)
    print(files_batch) 
    print(len(files_batch))
    # Read metadata of first file
    with rasterio.open(files_batch[0]) as src0: #open first files out of batch of 12 files
        meta = src0.meta

    # Update meta to reflect the number of layers in merged output geotiff file
    meta.update(count = len(files_batch)) #change output meta files as it will have 12 bands rather than 1 as in source files

    # Read each layer and write it to stack
    fnames = path +'/' + sub + '.tif'
    with rasterio.open(fnames, 'w', **meta) as dst: #output/destination(dst) file opened in write mode with an extra "meta" arbitrary keyword argument
        print(dst)
        for id, layer in enumerate(files_batch, start=1): # create list of 12 tuples 
            print(id,layer)
            with rasterio.open(layer) as src1:
                dst.write_band(id, src1.read(1)) #read 1 band at a time and write it to stacled tif

In [28]:
dirs = os.listdir(src_path)  #list of all classes
print(dirs)
print(len(dirs))

['Agro-forestry areas', 'Airports', 'Annual crops associated with permanent crops', 'Broad-leaved forest', 'Coastal lagoons', 'Complex cultivation patterns', 'Coniferous forest', 'Discontinuous urban fabric', 'Dump sites', 'Estuaries', 'Fruit trees and berry plantations', 'Green urban areas', 'Industrial or commercial units', 'Inland marshes', 'Intertidal flats', 'Mineral extraction sites', 'Mixed forest', 'Moors and heathland', 'Natural grassland', 'Non-irrigated arable land', 'Olive groves', 'Pastures', 'Peatbogs', 'Permanently irrigated land', 'Rice fields', 'Salt marshes', 'Sclerophyllous vegetation', 'Sparsely vegetated areas', 'Sport and leisure facilities', 'Transitional woodlandshrub', 'Water bodies', 'Vineyards']
32


In [ ]:
%%timeit
import glob2 # glob2 is faster than glob

for dir in dirs:
  sub_dirs = os.listdir(os.path.join(src_path,dir))
  path=os.path.join(src_path,dir)
  print(dir,len(sub_dirs))
  for data_folder in sub_dirs:
    path_tif_files=os.path.join(path,data_folder)
    merge_bands(path_tif_files,data_folder)

Agro-forestry areas 666
['/content/drive/MyDrive/BigEarthNet unique labelled/Agro-forestry areas/S2B_MSIL2A_20180515T112110_89_80/S2B_MSIL2A_20180515T112110_89_80_B01.tif', '/content/drive/MyDrive/BigEarthNet unique labelled/Agro-forestry areas/S2B_MSIL2A_20180515T112110_89_80/S2B_MSIL2A_20180515T112110_89_80_B02.tif', '/content/drive/MyDrive/BigEarthNet unique labelled/Agro-forestry areas/S2B_MSIL2A_20180515T112110_89_80/S2B_MSIL2A_20180515T112110_89_80_B03.tif', '/content/drive/MyDrive/BigEarthNet unique labelled/Agro-forestry areas/S2B_MSIL2A_20180515T112110_89_80/S2B_MSIL2A_20180515T112110_89_80_B04.tif', '/content/drive/MyDrive/BigEarthNet unique labelled/Agro-forestry areas/S2B_MSIL2A_20180515T112110_89_80/S2B_MSIL2A_20180515T112110_89_80_B05.tif', '/content/drive/MyDrive/BigEarthNet unique labelled/Agro-forestry areas/S2B_MSIL2A_20180515T112110_89_80/S2B_MSIL2A_20180515T112110_89_80_B06.tif', '/content/drive/MyDrive/BigEarthNet unique labelled/Agro-forestry areas/S2B_MSIL2A_2018